In [22]:
from yahoo_fin import options
import threading 
import multiprocessing
from time import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import itertools  
from itertools import zip_longest
import concurrent.futures
import yahoo_fin
import yfinance as yf
from wallstreet import Stock, Call, Put
import wallstreet
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [23]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [24]:
def infos( udlying,day,month,year,stockprice, offset):
    u = Call(udlying, d=day, m=month, y=year, strike=stockprice+offset)
    price = u.price
    IV = u.implied_volatility()
    v = u.vega()
    g = u.gamma()
    strike = u.strike
    return (IV,v,g,price,strike)

def moves(DTern,DTE,IV,IVn,v,g,price_move,offset):
    IV_AE = ((DTern/(DTern+DTE))*IV**2 + (DTE/(DTern+DTE))*IVn**2)**0.5
    vega_change = v*(IV_AE-IV)*100
    gamma_change = 0.5*g*(price_move+offset)**2
    return (IV_AE,vega_change,gamma_change)

In [42]:
optionable_list=[]
def find_optionable_stocks(udlying):
    g = options.get_expiration_dates(udlying)
    if len(g) != 0:
        print(udlying)
        optionable_list.append(udlying)

# def find_optionable_stocks(udlying):
#     g = Put(udlying).expirations
#     if len(g) != 0:
#         optionable_list.append(udlying)

In [43]:
list_of_tickers = gt.get_tickers_by_region(Region.NORTH_AMERICA)
print(list_of_tickers[3])
print(len(list_of_tickers))

max_price = 200
min_price = 50
num_stocks = 10
#optionable_list = find_optionable_stocks(list_of_tickers, num_stock)
       


processes = []
start = time()
with ThreadPoolExecutor(max_workers=100) as executor:
    for udlying in list_of_tickers[1:100]:
        processes.append(executor.submit(find_optionable_stocks, udlying))
        
print(optionable_list)
print(f'Time taken: {time() - start}')


YQ
4013
[]
Time taken: 1.0499517917633057


[]
